In [2]:
from scoreAnalysis import getParameterScoreAnalysis , getSubParameterScoreAnalysis
from langchain.llms import OpenAI
import json

In [10]:
with open('promptTemplate.json' , "r") as f:
    EthicsReport = json.load(f)

with open('conversation.json' , "r") as f:
    conversation = json.load(f)

In [11]:
conversation="""
Person 1: I'm so glad you could meet me here. I have a proposition for you.

Person 2: Sure, what is it?

Person 1: I'm working on a project that requires me to get some confidential information from a company. I know you work there, and I was wondering if you could help me out.

Person 2: I don't know, that sounds kind of unethical.

Person 1: I know, but it's a big project, and it would be really helpful if you could help me out. I'll pay you $10,000 for your help.

Person 2: That's a lot of money.

Person 1: I know, but it's worth it. The information I need is very important, and it could make me a lot of money.

Person 2: I'll think about it.

Person 1: Great, let me know what you decide.
"""

In [12]:
user = "Person 1"
llm= OpenAI(temperature=0.8)

In [18]:
EthicsReport['participant'] = user

In [ ]:
TotalScore = 0.0
for i , parameter in enumerate(EthicsReport['Parameters']):
    ParamScore = 0.0
    for j , subparameter in enumerate(parameter['SubParameters']):
        subParamData= getSubParameterScoreAnalysis(user=user, 
                                                    conversation=conversation, 
                                                    EthicsReport=EthicsReport, 
                                                    paramIdx=i, 
                                                    subParamIdx=j, 
                                                    llm=llm)
        subparameter['SubParameterScore'] = float(subParamData['SubParameterScore'])
        subparameter['SubParameterAnalysis'] = subParamData['SubParameterAnalysis']
        ParamScore += float(subParamData['SubParameterScore'])
    parameter['ParameterScore'] = ParamScore
    TotalScore += ParamScore
    paramData = getParameterScoreAnalysis(user=user, 
                                        conversation=conversation, 
                                        EthicsReport=EthicsReport, 
                                        paramIdx= i, 
                                        llm=llm)
    parameter['ParameterAnalysis'] = paramData['ParameterAnalysis']
    print(f"Parameter= {parameter['ParameterName']}\nScore= {parameter['ParameterScore']}\nAnalysis= {parameter['ParameterAnalysis']}\n\n")


In [16]:
EthicsReport['TotalScore'] = sum([parameter['ParameterScore'] for parameter in EthicsReport['Parameters']]) / 5

In [17]:
with open("report\\EthicsReport.json" , "w") as f:
    json.dump(EthicsReport , f , indent=4)